In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import librosa.display


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset

from torchsummary import summary
from tqdm import tqdm

In [2]:
sf = 8000
batch_size = 4
train_ratio = 0.8

In [3]:
def get_mfcc(data, sample_rate, window):
    mfccs_list = []
    for dat in data:
        mfccs = librosa.feature.mfcc(y=dat, sr=sample_rate, n_mfcc=window)
        mfccs = mfccs.T
        mfccs_list.append(mfccs)
    return mfccs_list

# Load HC

In [4]:
path = '/mnt/c/Users/user/Desktop/Roshidat/Workspace/PD_prediction/data/1_data/HC_AH'
dirc = os.listdir(path)

hc_data = []
for file in dirc:
    file_path = os.path.join(path, file)
    loaded_file, _ = librosa.load(file_path, sr=sf)
    hc_data.append(loaded_file)

len(hc_data)

41

In [5]:
hc_train_ratio =  int(len(hc_data) * train_ratio)
hc_train_ratio

32

In [6]:
y_hc = np.zeros(len(hc_data))  # Create an array of zeros with the same length as hc_data_padded
y_hc

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [7]:
hc_train = hc_data[:hc_train_ratio]
hc_test = hc_data[hc_train_ratio:]

y_hc_train = y_hc[:hc_train_ratio]
y_hc_test = y_hc[hc_train_ratio:]

len(hc_train), len(hc_test)

(32, 9)

# Load PD

In [8]:
pd_path = '/mnt/c/Users/user/Desktop/Roshidat/Workspace/PD_prediction/data/1_data/PD_AH'
pd_dirc = os.listdir(pd_path)

pd_data = []
for file in pd_dirc:
    file_path = os.path.join(pd_path, file)
    loaded_file, _ = librosa.load(file_path, sr=sf)
    pd_data.append(loaded_file)


len(pd_data)

40

In [9]:
y_pd = np.zeros(len(pd_data))+1  # Create an array of zeros with the same length as hc_data_padded
y_pd

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

In [10]:
pd_train_ratio =  int(len(pd_data) * train_ratio)

pd_train = pd_data[:pd_train_ratio]
pd_test = pd_data[pd_train_ratio:]

y_pd_train = y_pd[:pd_train_ratio]
y_pd_test = y_pd[pd_train_ratio:]

len(pd_train), len(pd_test) 

(32, 8)

In [11]:
X_train = hc_train + pd_train
y_train = np.concatenate((y_hc_train, y_pd_train))

X_test = hc_test + pd_test
y_test = np.concatenate((y_hc_test, y_pd_test))

In [12]:
len(X_train), len(y_train), len(X_test), len(y_test)

(64, 64, 17, 17)

In [13]:
X_train_mfcc = get_mfcc(X_train, sf, 13)
X_test_mfcc = get_mfcc(X_test, sf, 13)

In [14]:
len(X_train_mfcc), len(X_test_mfcc)

(64, 17)

In [17]:
X_train_mfcc[0].shape

(59, 13)

In [18]:
X_train_mfcc[1].shape

(36, 13)

In [19]:
type(X_train_mfcc)

list

In [20]:
class PDDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.data_size = len(data)
        


    def __len__(self):
        return self.data_size

    def __getitem__(self, idx):
        X = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.label[idx], dtype=torch.long)

        return X, y

In [21]:
train_ds = PDDataset(X_train_mfcc, y_train)
test_ds = PDDataset(X_test_mfcc, y_test)

In [22]:
dd = train_ds[60]
dd[0].shape, dd[1].shape

(torch.Size([45, 13]), torch.Size([]))

In [23]:
def _padding(batch):
    X, y = zip(*batch)
    X_padded = pad_sequence(X, batch_first=True, padding_value=0)

    return (
        X_padded,
        torch.stack(y),

    )

In [24]:
train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=_padding,
)
test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=_padding,
)

In [25]:
X_sample, y_sample = next(iter(train_loader))  # Check if the DataLoader works correctly

In [26]:
X_sample.shape, y_sample.shape

(torch.Size([4, 66, 13]), torch.Size([4]))

In [27]:
class PD_Classifier(nn.Module):
    def __init__(self, num_classes=2):
        super(PD_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        # Adaptive pooling to ensure fixed output size (e.g., 8x8)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((8, 8))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.adaptive_pool(x)  # Output shape: (batch, 32, 8, 8)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Example usage:
# model = PD_Classifier(num_classes=2)
# print(model)

In [28]:
model = PD_Classifier(num_classes=2)
model

PD_Classifier(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (adaptive_pool): AdaptiveAvgPool2d(output_size=(8, 8))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2048, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [29]:
# Training loop for PD_Classifier (single output neuron, sigmoid activation)
num_epochs = 10
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        inputs = inputs.unsqueeze(1)  # Add channel dimension for CNN (batch, channels, height, width)
        labels = labels.float().unsqueeze(1)  # Ensure shape (batch, 1) and float type
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        # acc = (y_pred.round() == y_batch).float().mean()
        #preds = (outputs > 0.5).float()
        preds = outputs.round()
        correct += (preds == labels).sum().item()
        
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

 10%|█         | 1/10 [00:00<00:01,  7.44it/s]

torch.Size([4, 1, 57, 13])
torch.Size([4, 1, 91, 13])
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 81, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 61, 13])
torch.Size([4, 1, 75, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 50, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 66, 13])
Epoch 1/10, Loss: 0.6091, Accuracy: 0.6562
torch.Size([4, 1, 75, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 65, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 53, 13])


 30%|███       | 3/10 [00:00<00:00,  8.88it/s]

torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 91, 13])
torch.Size([4, 1, 43, 13])
Epoch 2/10, Loss: 0.5389, Accuracy: 0.7344
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 45, 13])
torch.Size([4, 1, 81, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 91, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 52, 13])
torch.Size([4, 1, 58, 13])
torch.Size([4, 1, 61, 13])
torch.Size([4, 1, 75, 13])
Epoch 3/10, Loss: 0.4900, Accuracy: 0.7656
torch.Size([4, 1, 52, 13])
torch.Size([4, 1, 57, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 75, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 35, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 104, 13])


 40%|████      | 4/10 [00:00<00:00,  9.02it/s]

...

torch.Size([4, 1, 48, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 109, 13])
Epoch 6/10, Loss: 0.3506, Accuracy: 0.8750
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 53, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 56, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 75, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 61, 13])
torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 81, 13])
Epoch 7/10, Loss: 0.4608, Accuracy: 0.7812
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 86, 13])


 80%|████████  | 8/10 [00:00<00:00,  9.06it/s]

torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 61, 13])
torch.Size([4, 1, 42, 13])
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 58, 13])
torch.Size([4, 1, 57, 13])
torch.Size([4, 1, 91, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 81, 13])
torch.Size([4, 1, 99, 13])
Epoch 8/10, Loss: 0.3598, Accuracy: 0.8125
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 63, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 104, 13])
torch.Size([4, 1, 59, 13])
torch.Size([4, 1, 75, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 91, 13])
torch.Size([4, 1, 66, 13])
Epoch 9/10, Loss: 0.3328, Accuracy: 0.8594
torch.Size([4, 1, 109, 13])
torch.Size([4, 1, 59, 13])
torch.Size([4, 1, 53, 13])
torch.Size([4, 1, 86, 13])
torch.Size([4, 1, 99, 13])
torch.Size([4, 1, 61, 13])
torch.Size([4, 1, 8

100%|██████████| 10/10 [00:01<00:00,  9.36it/s]

torch.Size([4, 1, 53, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 66, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 68, 13])
torch.Size([4, 1, 62, 13])
torch.Size([4, 1, 86, 13])
Epoch 10/10, Loss: 0.3633, Accuracy: 0.8438


In [33]:
# Evaluation on test set
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        inputs = inputs.unsqueeze(1)  # Add channel dimension for CNN (batch, channels, height, width)
        labels = labels.float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        #preds = (outputs > 0.5).float()
        preds = outputs.round()
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)
test_loss = test_loss / test_total
test_acc = test_correct / test_total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.4f}')

Test Loss: 1.3237, Test Accuracy: 58.8235
